# 위험도 산출

In [22]:
import pandas as pd
from pandas import Series, DataFrame
import folium
from folium import plugins
import json
import geopandas as gpd
import numpy as np
from geoband.API import *
from folium.plugins import HeatMap
import matplotlib.pyplot as plt

In [23]:
import statsmodels.formula.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [24]:
psi = gpd.read_file('final_psi.geojson')

In [25]:
psi

gid  acci_cnt         pred          PSI     d_score  \
0     다바888170        63    64.035939    -1.035939   -1.035939   
1     다바889169        62   202.727389  -140.727389 -138.493621   
2     다바889171        58   754.266333  -696.266333 -641.007100   
3     다바911168        56   177.295008  -121.295008 -107.817785   
4     다바825181        55  1117.454335 -1062.454335 -927.539499   
...        ...       ...          ...          ...         ...   
6062  다바893188         1     1.751763    -0.751763   -0.011933   
6063  다바900257         1     1.867705    -0.867705   -0.013773   
6064  다바924165         1     1.873718    -0.873718   -0.013869   
6065  다바937135         1     1.774509    -0.774509   -0.012294   
6066  다바904126         1     2.434097    -1.434097   -0.022763   

                                               geometry  
0     MULTIPOLYGON (((127.37518 36.35034, 127.37518 ...  
1     MULTIPOLYGON (((127.37629 36.34944, 127.37629 ...  
2     MULTIPOLYGON (((127.37629 36.35125, 127.37629 ...  
3     MULTIPOLYGON (((127.40081 36.34856, 127.40081 ...  
4     MULTIPOLYGON (((127.30494 36.36017, 127.30494 ...  
...                                                 ...  
6062  MULTIPOLYGON (((127.38073 36.36658, 127.38072 ...  
6063  MULTIPOLYGON (((127.38844 36.42879, 127.38844 ...  
6064  MULTIPOLYGON (((127.41530 36.34587, 127.41530 ...  
6065  MULTIPOLYGON (((127.42982 36.31883, 127.42981 ...  
6066  MULTIPOLYGON (((127.39306 36.31069, 127.39306 ...  

[6067 rows x 6 columns]

In [26]:
d_score = psi[['gid','d_score','geometry']]

In [27]:
d_score_sort = d_score.sort_values(by='d_score', ascending=False)
d_score_sort_100 = d_score_sort.head(100)
d_score_sort_100

gid    d_score                                           geometry
34   다바848176  12.695248  MULTIPOLYGON (((127.33059 36.35570, 127.33058 ...
17   다바888169  12.437038  MULTIPOLYGON (((127.37518 36.34944, 127.37518 ...
36   다바844188   9.133789  MULTIPOLYGON (((127.32610 36.36651, 127.32610 ...
91   다바930139   5.670951  MULTIPOLYGON (((127.42201 36.32244, 127.42201 ...
94   다바938179   4.835610  MULTIPOLYGON (((127.43089 36.35850, 127.43089 ...
..        ...        ...                                                ...
345  다바941140   1.627336  MULTIPOLYGON (((127.43427 36.32334, 127.43427 ...
286  다바892178   1.625406  MULTIPOLYGON (((127.37962 36.35756, 127.37962 ...
160  다바846172   1.603796  MULTIPOLYGON (((127.32837 36.35209, 127.32836 ...
340  다바943168   1.589459  MULTIPOLYGON (((127.43648 36.34859, 127.43647 ...
175  다바950176   1.572375  MULTIPOLYGON (((127.44427 36.35580, 127.44427 ...

[100 rows x 3 columns]

In [11]:
final_100_d_score = gpd.GeoDataFrame(d_score_sort_100, geometry='geometry')
final_100_d_score.to_file('final_100_d_score.geojson', driver="GeoJSON",index=False)

In [12]:
final_100_d_score.to_csv('final_100_d_score.csv')#csv

In [13]:
center = [36.34, 127.39] #google맵 대전시 중심 좌표
m = folium.Map(location=center,zoom_start=11.5)

In [14]:
style = {'fillColor':'#f5f5f5', 'lineColor':'#ffffbf'}
style1 = {'fillColor': '#228B22', 'lineColor': '#228B22'}
style2 = {'fillColor': '#00FFFFFF', 'lineColor': '#00FFFFFF'}

In [15]:
folium.GeoJson(
    final_100_d_score,
    style_function=lambda x:style1
).add_to(m)
m

In [16]:
#지도를 html형식으로 저장
import folium
from IPython.display import display
m.save('d_score_map.html')

In [17]:
d_score_sort = d_score.sort_values(by='d_score', ascending=False)
d_score_sort_10 = d_score_sort.head(10)
d_score_sort_10

gid    d_score                                           geometry
34   다바848176  12.695248  MULTIPOLYGON (((127.33059 36.35570, 127.33058 ...
17   다바888169  12.437038  MULTIPOLYGON (((127.37518 36.34944, 127.37518 ...
36   다바844188   9.133789  MULTIPOLYGON (((127.32610 36.36651, 127.32610 ...
91   다바930139   5.670951  MULTIPOLYGON (((127.42201 36.32244, 127.42201 ...
94   다바938179   4.835610  MULTIPOLYGON (((127.43089 36.35850, 127.43089 ...
90   다바848177   4.697352  MULTIPOLYGON (((127.33058 36.35660, 127.33058 ...
120  다바893145   4.669721  MULTIPOLYGON (((127.38078 36.32781, 127.38078 ...
106  다바891186   4.639492  MULTIPOLYGON (((127.37850 36.36477, 127.37850 ...
144  다바891164   4.497940  MULTIPOLYGON (((127.37853 36.34494, 127.37853 ...
97   다바841112   4.347046  MULTIPOLYGON (((127.32291 36.29799, 127.32291 ...

In [18]:
final_10_d_score = gpd.GeoDataFrame(d_score_sort_10, geometry='geometry')

In [19]:
folium.GeoJson(
    final_10_d_score,
    style_function=lambda x:style1
).add_to(m)
m

In [20]:
sigon = gpd.read_file('25-Copy1.대전광역시_법정경계(시군구).geojson')

In [21]:
folium.GeoJson(
    final_10_d_score,
    style_function=lambda x:style1
).add_to(m)
folium.GeoJson(data=sigon["geometry"], style_function=lambda x:style1).add_to(m)
m